## **1) Setup Environment**

In [ ]:
!pip install torch opencv-python

## **2) Upload Images**

In [ ]:
from google.colab import files
import shutil

uploaded = files.upload()
for file in uploaded:
  shutil.move(file, 'ImageAI/Images')


Saving Adobe Scan 26-Oct-2021_1.jpg to Adobe Scan 26-Oct-2021_1.jpg
Saving IMG_20211010_132353.jpg to IMG_20211010_132353.jpg
Saving IMG_20211010_132405.jpg to IMG_20211010_132405.jpg


## **3) Predict**

In [ ]:
import argparse
import torch
import torchvision.models
import torchvision.transforms as transforms
import glob
from PIL import Image
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

addr=[]
val=[]
def prepare_image(image):
    if image.mode != 'RGB':
        image = image.convert("RGB")
    Transform = transforms.Compose([
            transforms.Resize([224,224]),      
            transforms.ToTensor(),
            ])
    image = Transform(image)   
    image = image.unsqueeze(0)
    return image.to(device)

def predict(image, model, item):
    image = prepare_image(image)
    with torch.no_grad():
        preds = model(image)
    print(item + "'s " r'rating is: %.2f' % preds.item())
    localValue = '%.2f' % preds.item()
    addr.append(item)
    val.append(localValue)

if __name__ == '__main__':
    files = glob.glob("ImageAI/Images/*.*")
    print("Please wait for the program to predict, It might take a while depending on the device's performance")
    for item in files: 
        parser = argparse.ArgumentParser()
        parser.add_argument('--image_path', type=str, default=item)
        config = parser.parse_args()
        image = Image.open(config.image_path)
        model = torchvision.models.resnet50()
        model.fc = torch.nn.Linear(in_features=2048, out_features=1)
        model.load_state_dict(torch.load('model/model-resnet50.pth', map_location=device)) 
        model.eval().to(device)
        predict(image, model, item)
    maxVal = max(val);
    index = val.index(maxVal)
    location = addr[index]
    print('The best image is '+'"'+ location + '"' + ' with a predicted rating of '+maxVal)
    im = Image.open(location)
    im.show()
    print('Enter any key to exit...')
    input()

Please wait for the program to predict, It might take a while depending on the device's performance


ValueError: ignored